### Setup

In [1]:
import networkx as nx
import random
import numpy as np
import itertools
from time import perf_counter
from math import ceil
from networkx.algorithms import bipartite

### Utility functions

In [2]:
def draw_edge_set(Gu, edges):
    colors = []
    for (u,v) in Gu.edges:
        if (u,v) in edges or (v,u) in edges:
            colors.append("red")
        else:
            colors.append("black")

    nx.draw_networkx(Gu, edge_color=colors)

In [3]:
def get_path_weight(G, path):
    total_weight = 0
    for u,v in zip(path, path[1:]):
        total_weight += G.edges[u,v]['weight']
    return total_weight

In [4]:
def contract(u,v):
    w1 = G.edges[u[-1],v[0]]['weight']
    w2 = G.edges[v[-1],u[0]]['weight']
    if w1 > w2:
        return u + v, w1
    else:
        return v + u, w2

In [5]:
def info(string, verbose):
    if verbose:
        print(f"%%% {string} %%%")

### Solvers

In [6]:
def via_brute_force(G, n, verbose=False):
    path = None
    w_max = 0
    for p in itertools.permutations(range(n)):
        w = get_path_weight(G, p)
        if w > w_max:
            path = p
            w_max = w
    return path

In [7]:
def via_matching(G, n, verbose=False):
    Gu = nx.complete_graph(n)
    for u,v in Gu.edges:
        w1 = G.edges[u,v]['weight']
        w2 = G.edges[v,u]['weight']
        Gu.edges[u,v]['weight'] = max(w1, w2)

    matching = nx.max_weight_matching(Gu)

    dir_matching = []
    for u,v in matching:
        if G.edges[u,v]['weight'] > G.edges[v,u]['weight']:
            dir_matching.append((u,v))
        else:
            dir_matching.append((v,u))
    # draw_edge_set(Gu, dir_matching)

    path = []
    for u,v in dir_matching:
        path.append(u)
        path.append(v)
    if n % 2 == 1:
        vertex_sum = n * (n-1) / 2
        final_v = int(vertex_sum - sum(path))
        path.append(final_v)
    
    return path

In [8]:
def via_iterated_matching(G, n, verbose=False):
    Gk = nx.complete_graph([(i,) for i in range(n)])
    for u in Gk.nodes:
        Gk.nodes[u]['value'] = u[0]
    for u,v in Gk.edges:
        _,w = contract(u,v)
        Gk.edges[u,v]['weight'] = w

    num_nodes = n
    while num_nodes > 1:
        info(f"num nodes: {num_nodes}", verbose)
        
        Gk_old = Gk
        Gk = nx.Graph()

        matching = nx.max_weight_matching(Gk_old)

        if num_nodes % 2 == 1:
            value_sum = n * (n-1) / 2
            for u,v in matching:
                value_sum -= Gk_old.nodes[u]['value'] + Gk_old.nodes[v]['value']
            final_value = int(value_sum)
            for u in Gk_old.nodes:
                if Gk_old.nodes[u]['value'] == final_value:
                    fluke = False
                    for x,y in matching:
                        if u in (x,y):
                            fluke = True
                    if not fluke:
                        Gk.add_node(u)
                        Gk.nodes[u]['value'] = final_value

        for u,v in matching:
            node,_ = contract(u,v)
            Gk.add_node(node)
            Gk.nodes[node]['value'] = Gk_old.nodes[u]['value'] + Gk_old.nodes[v]['value']

        for u in Gk.nodes:
            for v in Gk.nodes:
                if u != v:
                    _,w = contract(u,v)
                    Gk.add_edge(u, v, weight=w)

        num_nodes = len(Gk.nodes)

    for u in Gk.nodes:
        assert Gk.nodes[u]['value'] == n * (n-1) / 2

    return list(Gk.nodes)[0]

WARNING: NetworkX uses an assignment problem solver other than the Hungarian method

In [9]:
def via_cycle_cover(G, n, verbose=False):
    Gb = nx.Graph()
    for i in range(2*n):
        Gb.add_node(i)
    for u,v in G.edges:
        Gb.add_edge(u, v+n, weight=1-G.edges[u,v]['weight'])  # account for min weight NetworkX method

    mb_dict = bipartite.minimum_weight_full_matching(Gb, top_nodes=list(Gb.nodes)[:n])
    # draw_edge_set(Gb, list(mb_dict.items()))

    m_dict = {}
    for u,v in mb_dict.items():
        if u < n:
            x,y = u,v
        else:
            x,y = v,u
        m_dict[x] = y-n
    # draw_edge_set(Gu, list(m_dict.items()))


    cycle_cover = []

    done = set()
    cycle = []
    start = curr = 0

    while True:
        cycle.append(curr)
        done.add(curr)
        curr = m_dict[curr]
        if curr == start:
            cycle_cover.append(cycle)
            cycle = []
            flag = False
            for i in range(n):
                if i not in done:
                    start = curr = i
                    flag = True
                    break
            if not flag:
                break

    path_cover = []
    for cycle in cycle_cover:
        best_start = cycle[0]
        best_start_idx = 0
        min_weight = G.edges[cycle[-1],cycle[0]]['weight']
        for i, (u,v) in enumerate(zip(cycle, cycle[1:])):
            if G.edges[u,v]['weight'] < min_weight:
                best_start = v
                best_start_idx = i+1
                min_weight = G.edges[u,v]['weight']
        path = cycle[best_start_idx:] + cycle[:best_start_idx]
        path_cover.append(path)

    return list(itertools.chain(*path_cover))

In [10]:
def via_iterated_cycle_cover(G, n, verbose=False):
    Gk = nx.complete_graph([(i,) for i in range(n)], nx.DiGraph())
    num_nodes = n  
    
    while num_nodes > 1:
        info(f"num nodes: {num_nodes}", verbose)
        
        Gb = nx.Graph()
        for u in Gk.nodes:
            Gb.add_node(u)
            Gb.add_node(tuple([i + n for i in u]))
        for u,v in Gk.edges:
            Gb.add_edge(u, tuple([i + n for i in v]), weight=1-G.edges[u[-1],v[0]]['weight'])  # account for min weight NetworkX method
        
        mb_dict = bipartite.minimum_weight_full_matching(Gb, top_nodes=list(Gb.nodes)[::2])
        # draw_edge_set(Gb, list(mb_dict.items()))

        m_dict = {}
        for u,v in mb_dict.items():
            if u[0] < n:
                x,y = u,v
            else:
                x,y = v,u
            m_dict[x] = tuple([i - n for i in y])
        # draw_edge_set(Gu, list(m_dict.items()))


        cycle_cover = []

        done = set()
        cycle = []
        start = curr = random.choice([*m_dict])  # start with random node

        while True:
            cycle.extend(curr)
            done.add(curr)
            curr = m_dict[curr]
            if curr == start:
                cycle_cover.append(cycle)
                cycle = []
                flag = False
                for i in [*m_dict]:
                    if i not in done:
                        start = curr = i
                        flag = True
                        break
                if not flag:
                    break

        path_cover = []
        for cycle in cycle_cover:
            best_start = cycle[0]
            best_start_idx = 0
            min_weight = G.edges[cycle[-1],cycle[0]]['weight']
            for i, (u,v) in enumerate(zip(cycle, cycle[1:])):
                if G.edges[u,v]['weight'] < min_weight:
                    best_start = v
                    best_start_idx = i+1
                    min_weight = G.edges[u,v]['weight']
            path = cycle[best_start_idx:] + cycle[:best_start_idx]
            path_cover.append(tuple(path))
        
        Gk = nx.DiGraph()
        for u in path_cover:
            Gk.add_node(u)
        for u in Gk.nodes:
            for v in Gk.nodes:
                if u != v:
                    Gk.add_edge(u,v)
        num_nodes = len(Gk.nodes)

    return list(Gk.nodes)[0]

### Experiments

In [26]:
n = 700
G = nx.complete_graph(n, nx.DiGraph())
for (u,v) in G.edges:
    G.edges[u,v]['weight'] = random.random()

#### expected random path weight

In [27]:
(n-1)*0.5

349.5

#### theoretical upper bound on path weight

In [28]:
(n-1)*1

699

#### solvers

In [29]:
# solvers = [via_matching, via_iterated_matching, via_cycle_cover, via_iterated_cycle_cover]
solvers = [via_cycle_cover, via_iterated_cycle_cover]
verbose = True
sols = []

In [30]:
for func in solvers:
    print(f"solver: {func.__name__}")
    start = perf_counter()
    p = func(G, n, verbose=verbose)
    t = perf_counter() - start
    w = get_path_weight(G, p)
    sols.append((p, w, t))
    print(f"weight: {w}\ntime (s): {t}\n")

solver: via_cycle_cover
weight: 694.4985662493443
time (s): 4.3000039000000925

solver: via_iterated_cycle_cover
%%% num nodes: 700 %%%
%%% num nodes: 7 %%%
%%% num nodes: 2 %%%
weight: 696.5246047696877
time (s): 5.882794600000125

